In [1]:
from IPython import get_ipython
from IPython.display import display

In [2]:
pip install datasets transformers


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset


In [4]:
# Load the Omni-MATH dataset
dataset = load_dataset("KbsdJames/Omni-MATH")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# Convert the dataset to pandas DataFrames for train-test split
df = dataset['test'].to_pandas()
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert back to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
# Preprocess labels to ensure compatibility with single-label classification
def preprocess_labels(examples):
    examples["labels"] = int(examples["difficulty"])  # Map "difficulty" to "labels"
    return examples

train_dataset = train_dataset.map(preprocess_labels)
test_dataset = test_dataset.map(preprocess_labels)


Map:   0%|          | 0/3542 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["problem"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3542 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

In [9]:
# Load and configure the model for multi-class classification
config = AutoConfig.from_pretrained("distilbert-base-uncased", num_labels=10)  # Adjust `num_labels` based on your dataset
config.problem_type = "single_label_classification"

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-49585fc93ca3>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: xinerlewissa (xinerlewissa-university-of-sarawak-malaysia). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.530600,1.448306
2,1.391700,1.340811
3,1.136700,1.364611
4,1.020800,1.380213


TrainOutput(global_step=1772, training_loss=1.3736816251789206, metrics={'train_runtime': 747.8175, 'train_samples_per_second': 18.946, 'train_steps_per_second': 2.37, 'total_flos': 1877065864396800.0, 'train_loss': 1.3736816251789206, 'epoch': 4.0})

In [12]:
results = trainer.evaluate()
print(results)

{'eval_loss': 1.3802130222320557, 'eval_runtime': 12.9686, 'eval_samples_per_second': 68.319, 'eval_steps_per_second': 8.559, 'epoch': 4.0}
